In [1]:
%load_ext autoreload
%autoreload 2

import json
import os.path as osp
import os,sys
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.utils import to_undirected
from torch_cluster import radius_graph, knn_graph
from torch_geometric.datasets import MNISTSuperpixels
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
from tqdm import tqdm
import argparse
import warnings
warnings.simplefilter('ignore')
from time import strftime, gmtime

sys.path.append('../')
import utils
import model.net as net
import model.data_loader as data_loader
from evaluate import evaluate

In [2]:
pre_fix = '/export/home/phys/kyungmip/L1DeepMETv2/'

data_dir = pre_fix + 'data_ttbar/'        # name of the input data folder

batch_size = 32

In [3]:
dataloaders = data_loader.fetch_dataloader(data_dir = data_dir, batch_size=int(batch_size), validation_split=.2)

train_dl = dataloaders['train']
test_dl = dataloaders['test']

print('Training dataloader: {}, Test dataloader: {}'.format(len(train_dl), len(test_dl)))

Split: 199708
length of train/val data:  798834 199708
Training dataloader: 24964, Test dataloader: 6241


In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = str(0)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
genMET = np.array([])

with tqdm(total=len(train_dl)) as t:
    for data in train_dl:
        data = data.to(device)
        
        genMETx = data.y.detach().cpu()[:,0].numpy()
        genMETy = data.y.detach().cpu()[:,1].numpy()
        
        genMET = np.concatenate((genMET, np.sqrt(genMETx **2 + genMETy **2)), axis=0)
        
        print(genMET.shape)

  0%|          | 0/24964 [00:00<?, ?it/s]

(32,)
(64,)
(96,)
(128,)
(160,)
(192,)
(224,)
(256,)
(288,)
(320,)
(352,)
(384,)
(416,)
(448,)
(480,)
(512,)
(544,)
(576,)
(608,)
(640,)
(672,)
(704,)
(736,)
(768,)
(800,)
(832,)
(864,)
(896,)
(928,)
(960,)
(992,)
(1024,)
(1056,)
(1088,)
(1120,)
(1152,)
(1184,)
(1216,)
(1248,)
(1280,)
(1312,)
(1344,)
(1376,)
(1408,)
(1440,)
(1472,)
(1504,)
(1536,)
(1568,)
(1600,)
(1632,)
(1664,)
(1696,)
(1728,)
(1760,)
(1792,)
(1824,)
(1856,)
(1888,)
(1920,)
(1952,)
(1984,)
(2016,)
(2048,)
(2080,)
(2112,)
(2144,)
(2176,)
(2208,)
(2240,)
(2272,)
(2304,)
(2336,)
(2368,)
(2400,)
(2432,)
(2464,)
(2496,)
(2528,)
(2560,)
(2592,)
(2624,)
(2656,)
(2688,)
(2720,)
(2752,)
(2784,)
(2816,)
(2848,)
(2880,)
(2912,)
(2944,)
(2976,)
(3008,)
(3040,)
(3072,)
(3104,)
(3136,)
(3168,)
(3200,)
(3232,)
(3264,)
(3296,)
(3328,)
(3360,)
(3392,)
(3424,)
(3456,)
(3488,)
(3520,)
(3552,)
(3584,)
(3616,)
(3648,)
(3680,)
(3712,)
(3744,)
(3776,)
(3808,)
(3840,)
(3872,)
(3904,)
(3936,)
(3968,)
(4000,)
(4032,)
(4064,)
(4096,)
(4128,)
(4

(30464,)
(30496,)
(30528,)
(30560,)
(30592,)
(30624,)
(30656,)
(30688,)
(30720,)
(30752,)
(30784,)
(30816,)
(30848,)
(30880,)
(30912,)
(30944,)
(30976,)
(31008,)
(31040,)
(31072,)
(31104,)
(31136,)
(31168,)
(31200,)
(31232,)
(31264,)
(31296,)
(31328,)
(31360,)
(31392,)
(31424,)
(31456,)
(31488,)
(31520,)
(31552,)
(31584,)
(31616,)
(31648,)
(31680,)
(31712,)
(31744,)
(31776,)
(31808,)
(31840,)
(31872,)
(31904,)
(31936,)
(31968,)
(32000,)
(32032,)
(32064,)
(32096,)
(32128,)
(32160,)
(32192,)
(32224,)
(32256,)
(32288,)
(32320,)
(32352,)
(32384,)
(32416,)
(32448,)
(32480,)
(32512,)
(32544,)
(32576,)
(32608,)
(32640,)
(32672,)
(32704,)
(32736,)
(32768,)
(32800,)
(32832,)
(32864,)
(32896,)
(32928,)
(32960,)
(32992,)
(33024,)
(33056,)
(33088,)
(33120,)
(33152,)
(33184,)
(33216,)
(33248,)
(33280,)
(33312,)
(33344,)
(33376,)
(33408,)
(33440,)
(33472,)
(33504,)
(33536,)
(33568,)
(33600,)
(33632,)
(33664,)
(33696,)
(33728,)
(33760,)
(33792,)
(33824,)
(33856,)
(33888,)
(33920,)
(33952,)
(33984,)
(

(59840,)
(59872,)
(59904,)
(59936,)
(59968,)
(60000,)
(60032,)
(60064,)
(60096,)
(60128,)
(60160,)
(60192,)
(60224,)
(60256,)
(60288,)
(60320,)
(60352,)
(60384,)
(60416,)
(60448,)
(60480,)
(60512,)
(60544,)
(60576,)
(60608,)
(60640,)
(60672,)
(60704,)
(60736,)
(60768,)
(60800,)
(60832,)
(60864,)
(60896,)
(60928,)
(60960,)
(60992,)
(61024,)
(61056,)
(61088,)
(61120,)
(61152,)
(61184,)
(61216,)
(61248,)
(61280,)
(61312,)
(61344,)
(61376,)
(61408,)
(61440,)
(61472,)
(61504,)
(61536,)
(61568,)
(61600,)
(61632,)
(61664,)
(61696,)
(61728,)
(61760,)
(61792,)
(61824,)
(61856,)
(61888,)
(61920,)
(61952,)
(61984,)
(62016,)
(62048,)
(62080,)
(62112,)
(62144,)
(62176,)
(62208,)
(62240,)
(62272,)
(62304,)
(62336,)
(62368,)
(62400,)
(62432,)
(62464,)
(62496,)
(62528,)
(62560,)
(62592,)
(62624,)
(62656,)
(62688,)
(62720,)
(62752,)
(62784,)
(62816,)
(62848,)
(62880,)
(62912,)
(62944,)
(62976,)
(63008,)
(63040,)
(63072,)
(63104,)
(63136,)
(63168,)
(63200,)
(63232,)
(63264,)
(63296,)
(63328,)
(63360,)
(

(89344,)
(89376,)
(89408,)
(89440,)
(89472,)
(89504,)
(89536,)
(89568,)
(89600,)
(89632,)
(89664,)
(89696,)
(89728,)
(89760,)
(89792,)
(89824,)
(89856,)
(89888,)
(89920,)
(89952,)
(89984,)
(90016,)
(90048,)
(90080,)
(90112,)
(90144,)
(90176,)
(90208,)
(90240,)
(90272,)
(90304,)
(90336,)
(90368,)
(90400,)
(90432,)
(90464,)
(90496,)
(90528,)
(90560,)
(90592,)
(90624,)
(90656,)
(90688,)
(90720,)
(90752,)
(90784,)
(90816,)
(90848,)
(90880,)
(90912,)
(90944,)
(90976,)
(91008,)
(91040,)
(91072,)
(91104,)
(91136,)
(91168,)
(91200,)
(91232,)
(91264,)
(91296,)
(91328,)
(91360,)
(91392,)
(91424,)
(91456,)
(91488,)
(91520,)
(91552,)
(91584,)
(91616,)
(91648,)
(91680,)
(91712,)
(91744,)
(91776,)
(91808,)
(91840,)
(91872,)
(91904,)
(91936,)
(91968,)
(92000,)
(92032,)
(92064,)
(92096,)
(92128,)
(92160,)
(92192,)
(92224,)
(92256,)
(92288,)
(92320,)
(92352,)
(92384,)
(92416,)
(92448,)
(92480,)
(92512,)
(92544,)
(92576,)
(92608,)
(92640,)
(92672,)
(92704,)
(92736,)
(92768,)
(92800,)
(92832,)
(92864,)
(

...

(301888,)
(301920,)
(301952,)
(301984,)
(302016,)
(302048,)
(302080,)
(302112,)
(302144,)
(302176,)
(302208,)
(302240,)
(302272,)
(302304,)
(302336,)
(302368,)
(302400,)
(302432,)
(302464,)
(302496,)
(302528,)
(302560,)
(302592,)
(302624,)
(302656,)
(302688,)
(302720,)
(302752,)
(302784,)
(302816,)
(302848,)
(302880,)
(302912,)
(302944,)
(302976,)
(303008,)
(303040,)
(303072,)
(303104,)
(303136,)
(303168,)
(303200,)
(303232,)
(303264,)
(303296,)
(303328,)
(303360,)
(303392,)
(303424,)
(303456,)
(303488,)
(303520,)
(303552,)
(303584,)
(303616,)
(303648,)
(303680,)
(303712,)
(303744,)
(303776,)
(303808,)
(303840,)
(303872,)
(303904,)
(303936,)
(303968,)
(304000,)
(304032,)
(304064,)
(304096,)
(304128,)
(304160,)
(304192,)
(304224,)
(304256,)
(304288,)
(304320,)
(304352,)
(304384,)
(304416,)
(304448,)
(304480,)
(304512,)
(304544,)
(304576,)
(304608,)
(304640,)
(304672,)
(304704,)
(304736,)
(304768,)
(304800,)
(304832,)
(304864,)
(304896,)
(304928,)
(304960,)
(304992,)
(305024,)
(305056,)


(328224,)
(328256,)
(328288,)
(328320,)
(328352,)
(328384,)
(328416,)
(328448,)
(328480,)
(328512,)
(328544,)
(328576,)
(328608,)
(328640,)
(328672,)
(328704,)
(328736,)
(328768,)
(328800,)
(328832,)
(328864,)
(328896,)
(328928,)
(328960,)
(328992,)
(329024,)
(329056,)
(329088,)
(329120,)
(329152,)
(329184,)
(329216,)
(329248,)
(329280,)
(329312,)
(329344,)
(329376,)
(329408,)
(329440,)
(329472,)
(329504,)
(329536,)
(329568,)
(329600,)
(329632,)
(329664,)
(329696,)
(329728,)
(329760,)
(329792,)
(329824,)
(329856,)
(329888,)
(329920,)
(329952,)
(329984,)
(330016,)
(330048,)
(330080,)
(330112,)
(330144,)
(330176,)
(330208,)
(330240,)
(330272,)
(330304,)
(330336,)
(330368,)
(330400,)
(330432,)
(330464,)
(330496,)
(330528,)
(330560,)
(330592,)
(330624,)
(330656,)
(330688,)
(330720,)
(330752,)
(330784,)
(330816,)
(330848,)
(330880,)
(330912,)
(330944,)
(330976,)
(331008,)
(331040,)
(331072,)
(331104,)
(331136,)
(331168,)
(331200,)
(331232,)
(331264,)
(331296,)
(331328,)
(331360,)
(331392,)


(354688,)
(354720,)
(354752,)
(354784,)
(354816,)
(354848,)
(354880,)
(354912,)
(354944,)
(354976,)
(355008,)
(355040,)
(355072,)
(355104,)
(355136,)
(355168,)
(355200,)
(355232,)
(355264,)
(355296,)
(355328,)
(355360,)
(355392,)
(355424,)
(355456,)
(355488,)
(355520,)
(355552,)
(355584,)
(355616,)
(355648,)
(355680,)
(355712,)
(355744,)
(355776,)
(355808,)
(355840,)
(355872,)
(355904,)
(355936,)
(355968,)
(356000,)
(356032,)
(356064,)
(356096,)
(356128,)
(356160,)
(356192,)
(356224,)
(356256,)
(356288,)
(356320,)
(356352,)
(356384,)
(356416,)
(356448,)
(356480,)
(356512,)
(356544,)
(356576,)
(356608,)
(356640,)
(356672,)
(356704,)
(356736,)
(356768,)
(356800,)
(356832,)
(356864,)
(356896,)
(356928,)
(356960,)
(356992,)
(357024,)
(357056,)
(357088,)
(357120,)
(357152,)
(357184,)
(357216,)
(357248,)
(357280,)
(357312,)
(357344,)
(357376,)
(357408,)
(357440,)
(357472,)
(357504,)
(357536,)
(357568,)
(357600,)
(357632,)
(357664,)
(357696,)
(357728,)
(357760,)
(357792,)
(357824,)
(357856,)


(381280,)
(381312,)
(381344,)
(381376,)
(381408,)
(381440,)
(381472,)
(381504,)
(381536,)
(381568,)
(381600,)
(381632,)
(381664,)
(381696,)
(381728,)
(381760,)
(381792,)
(381824,)
(381856,)
(381888,)
(381920,)
(381952,)
(381984,)
(382016,)
(382048,)
(382080,)
(382112,)
(382144,)
(382176,)
(382208,)
(382240,)
(382272,)
(382304,)
(382336,)
(382368,)
(382400,)
(382432,)
(382464,)
(382496,)
(382528,)
(382560,)
(382592,)
(382624,)
(382656,)
(382688,)
(382720,)
(382752,)
(382784,)
(382816,)
(382848,)
(382880,)
(382912,)
(382944,)
(382976,)
(383008,)
(383040,)
(383072,)
(383104,)
(383136,)
(383168,)
(383200,)
(383232,)
(383264,)
(383296,)
(383328,)
(383360,)
(383392,)
(383424,)
(383456,)
(383488,)
(383520,)
(383552,)
(383584,)
(383616,)
(383648,)
(383680,)
(383712,)
(383744,)
(383776,)
(383808,)
(383840,)
(383872,)
(383904,)
(383936,)
(383968,)
(384000,)
(384032,)
(384064,)
(384096,)
(384128,)
(384160,)
(384192,)
(384224,)
(384256,)
(384288,)
(384320,)
(384352,)
(384384,)
(384416,)
(384448,)


(407872,)
(407904,)
(407936,)
(407968,)
(408000,)
(408032,)
(408064,)
(408096,)
(408128,)
(408160,)
(408192,)
(408224,)
(408256,)
(408288,)
(408320,)
(408352,)
(408384,)
(408416,)
(408448,)
(408480,)
(408512,)
(408544,)
(408576,)
(408608,)
(408640,)
(408672,)
(408704,)
(408736,)
(408768,)
(408800,)
(408832,)
(408864,)
(408896,)
(408928,)
(408960,)
(408992,)
(409024,)
(409056,)
(409088,)
(409120,)
(409152,)
(409184,)
(409216,)
(409248,)
(409280,)
(409312,)
(409344,)
(409376,)
(409408,)
(409440,)
(409472,)
(409504,)
(409536,)
(409568,)
(409600,)
(409632,)
(409664,)
(409696,)
(409728,)
(409760,)
(409792,)
(409824,)
(409856,)
(409888,)
(409920,)
(409952,)
(409984,)
(410016,)
(410048,)
(410080,)
(410112,)
(410144,)
(410176,)
(410208,)
(410240,)
(410272,)
(410304,)
(410336,)
(410368,)
(410400,)
(410432,)
(410464,)
(410496,)
(410528,)
(410560,)
(410592,)
(410624,)
(410656,)
(410688,)
(410720,)
(410752,)
(410784,)
(410816,)
(410848,)
(410880,)
(410912,)
(410944,)
(410976,)
(411008,)
(411040,)


In [ ]:
genMET

### Plot MET

In [ ]:
genMET = np.array(genMET)
genMET_binned = []

#nbins = 500
#binnings = np.linspace(0, 1000, num=nbins+1)

#binnings = [0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 240, 300, 400, 1000]
binnings = [0, 20, 40, 60, 80, 100, 120, 140, 160, 1000]

for idx in range(len(binnings)-1):
    genMET_binned.append( np.logical_and(genMET > binnings[idx], genMET < binnings[idx+1]) )

In [ ]:
genMET_binned_entries = [sum(genMET_binned[i]) for i in range(len(genMET_binned))]

In [ ]:
genMET_binned_entries

In [ ]:
nEntries = sum(genMET_binned_entries)

In [ ]:
per_genMET_bin_weight = [ nEntries/entry for entry in genMET_binned_entries]

In [ ]:
per_genMET_bin_weight = per_genMET_bin_weight / sum(per_genMET_bin_weight)

In [ ]:
per_genMET_bin_weight